In [1]:
import re

In [2]:
with open('input') as f:
    data = f.read().strip().split('\n\n')

In [3]:
def parse_input(entry):

    monkey_line, item_line, operation_line, test_line, true_line, false_line = [
        e.strip() for e in entry.split('\n')]
    
    monkey_id = int(re.search(r'Monkey (\d+):', monkey_line).group(1))
    
    item_ids = [
        int(item) for item 
        in re.search(r'Starting items: ([\d, ]+)', item_line).group(1).split(', ')]
    
    o_match = re.search(r'Operation: new = old ([\*\+]) (\w+)', operation_line)
    operator, operand = o_match.group(1), o_match.group(2)
    operand = int(operand) if operand != 'old' else operand

    if operator == '+':
        operation = (lambda x: x + operand) if type(operand) == int else (lambda x: x + x)
    else: 
        operation = (lambda x: x * operand) if type(operand) == int else (lambda x: x * x)
    
    divisor = int(re.search(r'Test: divisible by (\d+)', test_line).group(1))
    true_target = int(re.search(r'If true: throw to monkey (\d+)', true_line).group(1))
    false_target = int(re.search(r'If false: throw to monkey (\d+)', false_line).group(1))
    
    return monkey_id, item_ids, operation, divisor, true_target, false_target

# Part 1

In [4]:
class Monkey():
    def __init__(self, monkey_id, current_items, operation, divisor, true_target, false_target):

        self.id = monkey_id
        self.current_items = current_items
        self.operation = operation
        self.divisor = divisor
        self.true_target = true_target
        self.false_target = false_target

        self.item_counter = 0

    def turn(self, other_monkeys):
        for worry_level in self.current_items:
            worry_level = self.operation(worry_level)
            worry_level = worry_level // 3
            if worry_level % self.divisor == 0:
                other_monkeys[self.true_target].current_items.append(worry_level)
            else: 
                other_monkeys[self.false_target].current_items.append(worry_level)
            self.item_counter += 1
        self.current_items = []


monkeys = []

for entry in data:
    monkey_id, current_items, operation, divisor, true_target, false_target = parse_input(entry)
    monkeys.append(
        Monkey(monkey_id, current_items, operation, divisor, true_target, false_target)
    )

In [5]:
for i in range(20):

    for monkey in monkeys: 
        monkey.turn(other_monkeys=monkeys)

most_active = sorted(monkeys, key=lambda x: x.item_counter, reverse=True)

for monkey in most_active:
    print(f'Monkey {monkey.id} inspected items {monkey.item_counter} times.')

most_active[0].item_counter * most_active[1].item_counter

Monkey 4 inspected items 226 times.
Monkey 3 inspected items 222 times.
Monkey 6 inspected items 194 times.
Monkey 2 inspected items 134 times.
Monkey 1 inspected items 127 times.
Monkey 5 inspected items 93 times.
Monkey 7 inspected items 47 times.
Monkey 0 inspected items 20 times.


50172

# Part 2

In [6]:
class Monkey():
    def __init__(self, monkey_id, current_items, operation, divisor, true_target, false_target):

        self.id = monkey_id
        self.current_items = current_items
        self.operation = operation
        self.divisor = divisor
        self.true_target = true_target
        self.false_target = false_target

        self.item_counter = 0

        # common multiple of the test divisors of all monkeys
        self.common_multiple = 0  # init now, set later

    def turn(self, other_monkeys):
        for worry_level in self.current_items:
            worry_level = self.operation(worry_level)
            test_remainder = worry_level % self.divisor
            # "modulo trick": 
            # pass the remainder of worry_level / common_multiple (of all monkeys) 
            # instead of the worry_level itself
            if test_remainder == 0:
                other_monkeys[self.true_target].current_items.append(worry_level % self.common_multiple)
            else: 
                other_monkeys[self.false_target].current_items.append(worry_level % self.common_multiple)
            self.item_counter += 1
        self.current_items = []


monkeys = []

for entry in data:
    monkey_id, current_items, operation, divisor, true_target, false_target = parse_input(entry)
    monkeys.append(
        Monkey(monkey_id, current_items, operation, divisor, true_target, false_target)
    )

In [7]:
from math import prod

# determine common multiple of all test divisors
# and set als Monkey.common_multiple for all monkeys

common_multiple = prod([m.divisor for m in monkeys])

for m in monkeys:
    m.common_multiple = common_multiple

In [8]:
from tqdm import tqdm

for i in tqdm(range(10000)):

    for monkey in monkeys: 
        monkey.turn(other_monkeys=monkeys)

most_active = sorted(monkeys, key=lambda x: x.item_counter, reverse=True)

for monkey in most_active:
    print(f'Monkey {monkey.id} inspected items {monkey.item_counter} times.')

most_active[0].item_counter * most_active[1].item_counter

100%|██████████| 10000/10000 [00:00<00:00, 46932.16it/s]

Monkey 3 inspected items 109098 times.
Monkey 4 inspected items 106461 times.
Monkey 2 inspected items 73630 times.
Monkey 1 inspected items 70776 times.
Monkey 5 inspected items 62841 times.
Monkey 6 inspected items 61442 times.
Monkey 7 inspected items 50365 times.
Monkey 0 inspected items 10659 times.


11614682178